In [30]:
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

def load_model_and_scaler(model_path, scaler_path):
    """Load the trained K-Means model and StandardScaler."""
    kmeans = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    return kmeans, scaler

def calculate_user_points(monthly_spending, monthly_savings, last_month_savings):
    """Compute the derived features, normalize, and calculate user points."""
    if monthly_savings == 0:
        monthly_savings = 0.01

    if last_month_savings == 0:
        last_month_savings = 0.01

    # Derived features
    withdrawal_ratio = monthly_spending / monthly_savings
    savings_growth_rate = (monthly_savings - last_month_savings) / last_month_savings

    # Feature weights
    weights = {
        "Monthly_Savings": 0.5,
        "Monthly_Spending": 0.2,
        "Withdrawal_Ratio": -0.2,
        "Savings_Growth_Rate": 0.2,
    }

    # Normalize features (assuming max and min values from training data)
    max_savings, max_spending = 9904865, 72944  # Adjust based on dataset
    max_withdrawal_ratio, min_savings_growth_rate, max_savings_growth_rate = 5, -1, 2

    norm_savings = monthly_savings / max_savings
    norm_spending = monthly_spending / max_spending
    norm_withdrawal_ratio = 1 - (withdrawal_ratio / max_withdrawal_ratio)
    norm_savings_growth = (savings_growth_rate - min_savings_growth_rate) / (max_savings_growth_rate - min_savings_growth_rate)

    # Compute the final score
    points = (
        norm_savings * weights["Monthly_Savings"] +
        norm_spending * weights["Monthly_Spending"] +
        norm_withdrawal_ratio * weights["Withdrawal_Ratio"] +
        norm_savings_growth * weights["Savings_Growth_Rate"]
    )

    # Scale points to a range (0 to 1000)
    points = max(0, min(1000, points * 1000))  # Ensure points stay in range
    return round(points, 2)

def predict_cluster(monthly_spending, monthly_savings, last_month_savings, model_path, scaler_path):
    """Calculate points and predict cluster."""
    kmeans, scaler = load_model_and_scaler(model_path, scaler_path)
    points = calculate_user_points(monthly_spending, monthly_savings, last_month_savings)
    points_scaled = scaler.transform(np.array([[points]]))  # Reshape for model
    cluster = kmeans.predict(points_scaled)[0]
    return cluster, points

def cluster_Mapping(cluster):
    if cluster == 0:
        return 0
    elif cluster == 1:
        return 2
    elif cluster == 2:
        return 1
    elif cluster == 3:
        return 4
    elif cluster == 4:
        return 3

# Example usage:
if __name__ == "__main__":
    model_path = "/content/kmeans_model.pkl"
    scaler_path = "/content/scaler.pkl"

    user_cluster, user_points = predict_cluster(700, 100, 7200, model_path, scaler_path)
    user_cluster = cluster_Mapping(user_cluster)
    print(f"User belongs to cluster {user_cluster} with {user_points} points.")


User belongs to cluster 0 with 82.85 points.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
